### Preparar ambiente

In [1]:
import enum
print(enum.__file__)  
# standard library location should be something like 
# /usr/local/lib/python3.6/enum.py

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/enum.py


In [2]:
import sys
!{sys.executable} -m pip uninstall -y enum34

In [3]:
!{sys.executable} -m pip install awswrangler

In [4]:
!{sys.executable} -m pip install geopandas

In [5]:
!{sys.executable} -m pip install PyAthena

### Conectar con pyathena

In [6]:
from pyathena.pandas.cursor import PandasCursor
from pyathena import connect
import pandas as pd
import time
# agregar directorio en el cual se almacenan las tablas que se van a consultar
directorio = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/athena-results/'
# base de datos
bd = 'graphdata' # Database in Glue
cursor = connect(s3_staging_dir = directorio, region_name = 'us-east-1', schema_name = bd, cursor_class = PandasCursor).cursor()

In [7]:
import pandas as pd
import awswrangler as wr
import numpy as np
import calendar
import boto3
s3 = boto3.resource('s3')

In [8]:
import geopandas as gpd
import glob
import os

### Verificar que están localmente los archivos Shapefile. Si no están, descargarlos

In [9]:
if not os.path.exists('Shapefiles'):
    bucket_shapefiles = 'iadbprod-csd-hub-analyticaldata'
    folder_local = 'Shapefiles'
    os.mkdir(folder_local)
    for key in s3.Bucket(bucket_shapefiles).objects.all().filter(Prefix=folder_local):
        data_key = key.key
        directorios = data_key.split('/')
        if len(directorios) == 3 and not 'Municipios' in data_key: # carpeta originen, pais y archivo (shp, cpg, csv, dbf, prj, shx)
            folder_pais = os.path.join(folder_local, directorios[1])
            if not os.path.exists(folder_pais):
                os.mkdir(folder_pais)
            new_data_key = os.path.join(folder_pais,directorios[2])
            s3.Bucket(bucket_shapefiles).download_file(data_key, new_data_key)

### Seleccionar shapefiles de paises especificos
##### borrar carpetas en terminal / rm -rf "shapefiles2"

### Actualización deltas -  Aca comienza el flujo

In [10]:
fecha_inicio = '09-01-2020'
fecha_final = '09-30-2020'
fechas = pd.date_range(fecha_inicio, fecha_final)

In [11]:
historico_hogares = f'historico_hogares'
for fecha in fechas:
    start_time_day = time.time()
    day = str(fecha.day)
    if len(day) == 1:
        day = f'0{day}'
    codigo_fecha = str(fecha.month) + day
    print(codigo_fecha)
    
    
    # Generando tabla temporal de coordenadas de hogates
    # ==============================
    todos_usuarios_delta = f'todos_usuarios_delta_dist_corregida_{codigo_fecha}'
    print(todos_usuarios_delta)
    starttime=time.time()
    query = f'''SELECT delta.lat_final as lat_hogar,
                       delta.long_final as long_hogar,
                       delta.iso_country_code  
                FROM (SELECT filtro.caid, 
                             filtro.n_obs, 
                             filtro.lat_final, 
                             filtro.long_final, 
                             t2.id_provincia, 
                             t2.provincia, 
                             filtro.year, 
                             filtro.month, 
                             filtro.day, 
                             filtro.iso_country_code 
                     FROM {todos_usuarios_delta} AS filtro 
                     LEFT JOIN (SELECT ROUND(lat_hogar,3) AS lat_hogar2, 
                                       ROUND(long_hogar,3) AS long_hogar2, 
                                       id_provincia, 
                                       provincia, 
                                       iso_country_code 
                                FROM {historico_hogares} 
                                GROUP BY ROUND(lat_hogar,3),
                                         ROUND(long_hogar,3),
                                         id_provincia,
                                         provincia,
                                         iso_country_code) AS t2  
                    ON filtro.lat_final=t2.lat_hogar2 
                    AND filtro.long_final=t2.long_hogar2 
                    AND filtro.iso_country_code=t2.iso_country_code) AS delta 
                WHERE delta.id_provincia IS NULL 
                GROUP BY delta.lat_final,
                         delta.long_final,
                         delta.iso_country_code'''
    df_usuario_coord = cursor.execute(query).as_pandas()    
    print(f"Tiempo query select: {int((time.time()-starttime))}s")
    print(f"Tamaño tabla sin id_provincia en historico: {df_usuario_coord.shape[0]}")
    #assert df_usuario_coord[df_usuario_coord.duplicated(['long_hogar','lat_hogar','iso_country_code'])].shape[0] == 0
    
    
    # Asignando region a la tabla hogares por dia
    # ==============================
    starttime=time.time()
    paises = ['AR','CO','PE','NI','SV','BO','DO','CL','CR','PY','BZ','HN','JM','TT','UY','GY','EC','GT','PA','VE',"BR","MX"]
    for pais in paises:
        print(f"Pais: {pais}")
        df_usuario_coord2 = df_usuario_coord.loc[df_usuario_coord["iso_country_code"]==pais]
        print(df_usuario_coord2.shape)
        gdf_homes = gpd.GeoDataFrame(df_usuario_coord2,
                                     geometry=gpd.points_from_xy(
                                         df_usuario_coord2.long_hogar, 
                                         df_usuario_coord2.lat_hogar), 
                                         crs = 'epsg:4326')
        gdf_homes = gdf_homes.dropna(subset=['long_hogar','lat_hogar'])
        path = glob.glob(f'Shapefiles/{pais}/*.shp')
        mapa = gpd.read_file(path[0],encoding = 'utf-8')
        mapa = mapa.to_crs("epsg:4326")
        if pais == "CL":
            mapa = mapa.loc[:,['codregion', 'Region', 'geometry']]
            gdf_homes = gpd.sjoin(gdf_homes, mapa, op='within') 
            gdf_homes=gdf_homes.loc[:,['long_hogar','lat_hogar','codregion', 'Region','iso_country_code']]
            gdf_homes=gdf_homes.rename(columns={"codregion":"id_provincia",'Region': 'Provincia'})
            gdf_homes=gdf_homes.dropna(subset=["id_provincia"])
        if pais == "CR":       
            mapa = mapa.loc[:,['COD_PROV', 'NPROVINCIA', 'geometry']]
            gdf_homes = gpd.sjoin(gdf_homes, mapa, op='within') 
            gdf_homes=gdf_homes.loc[:,['long_hogar','lat_hogar','COD_PROV', 'NPROVINCIA','iso_country_code']]
            gdf_homes=gdf_homes.rename(columns={"COD_PROV":"id_provincia",'NPROVINCIA': 'Provincia'})
            gdf_homes=gdf_homes.dropna(subset=["id_provincia"])
        if ((pais != "CR") & (pais != "CL")):
            mapa = mapa.loc[:,["ID_1",'NAME_1','geometry']]
            gdf_homes = gpd.sjoin(gdf_homes, mapa, op='within') 
            gdf_homes=gdf_homes.loc[:,['long_hogar','lat_hogar',"ID_1",'NAME_1','iso_country_code']]
            gdf_homes=gdf_homes.rename(columns={"ID_1":"id_provincia",'NAME_1': 'Provincia'})
            gdf_homes=gdf_homes.dropna(subset=["id_provincia"])
        if pais == 'AR':
            base_nuevos=gdf_homes
        else:
            base_nuevos=base_nuevos.append(gdf_homes) 
    #df a nivel nacion   
    print(f"Asociar id_provincia a hogares: {int(time.time()-starttime)}s")
    
    
    # Creando tabla hogares final por dia
    # ==============================
    starttime = time.time()
    wr.s3.to_parquet(  # Storing the data and metadata to Data Lake
        df=base_nuevos,
        dataset=True,
        table=f"base_nuevos_{codigo_fecha}",
        database="graphdata",
        dtype={'Provincia': 'string'},
        path=f"s3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Tablas_hogares/Tablas_hogares_final/base_nuevos_{codigo_fecha}"
    )
    print(f"Guardar tabla base_nuevos_dia:{int((time.time()-starttime))}s")    
    
    
    # Creando tabla historica a nivel de hogares
    # ==============================
    starttime = time.time()
    historico_hogares = f'historico_hogares'
    tabla_hogares_fecha = f'base_nuevos_{codigo_fecha}'
    if codigo_fecha == '201': 
        # CREAR TABLA
        query = f'''CREATE TABLE {historico_hogares} 
            WITH (    
             external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/{historico_hogares}',  
             format = 'PARQUET', 
             parquet_compression = 'SNAPPY') AS 
            SELECT *
            FROM {tabla_hogares_fecha}  '''
        cursor.execute(query)
    else:
        # INSERTAR
        query = f'''INSERT INTO {historico_hogares} SELECT * FROM {tabla_hogares_fecha} '''
        cursor.execute(query) 
    print(f"Tiempo total dia {fecha}: {int(time.time() - start_time_day)}s")
    print("*"*90)
    query = (f"MSCK REPAIR TABLE historico_hogares")
    cursor.execute(query) 
    time.sleep(60)
    
    
    # Creando tabla usuarios final por dia
    # ==============================
    starttime=time.time()
    tabla_final_usuario_dia = f'usuarios_final_{codigo_fecha}'
    query = f'''CREATE TABLE {tabla_final_usuario_dia} 
         WITH (    
         external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-temporal/Tablas_usuarios/Usuarios_final/{tabla_final_usuario_dia}',  
         format = 'PARQUET', 
         parquet_compression = 'SNAPPY') AS 
         SELECT filtro.caid,
                filtro.year,
                filtro.month,
                filtro.day,
                filtro.distancia_recorrida,
                filtro.n_obs,
                filtro.n_obs_noche,
                filtro.n_obs_madrugada,
                filtro.lat_hogar,
                filtro.long_hogar,
                filtro.n_obs_hogar,
                filtro.lat_mas_visitado,
                filtro.long_mas_visitado,
                filtro.n_obs_mas_visitado,
                filtro.lat_frecuente,
                filtro.long_frecuente,
                filtro.lat_final,
                filtro.long_final,
                t2.id_provincia, 
                t2.provincia,
                filtro.iso_country_code
        FROM {todos_usuarios_delta} AS filtro 
            LEFT JOIN (SELECT ROUND(lat_hogar,3) AS lat_hogar2, 
                              ROUND(long_hogar,3) AS long_hogar2, 
                              id_provincia, 
                              provincia, 
                              iso_country_code 
                       FROM historico_hogares 
                       GROUP BY ROUND(lat_hogar,3),
                                ROUND(long_hogar,3),
                                id_provincia,
                                provincia,
                                iso_country_code) AS t2  
            ON filtro.lat_final=t2.lat_hogar2 
            AND filtro.long_final=t2.long_hogar2 
            AND filtro.iso_country_code=t2.iso_country_code'''
    cursor.execute(query) 
    print(f"Crear tabla final del dia con hogares e id_provincia: {int((time.time()-starttime))}s")
    time.sleep(60)
    
    
    # Creando tabla historico a nivel de usuarios
    # ==============================
    historico_usuarios = f'historico_usuarios'
    if codigo_fecha == '201': 
        # CREAR TABLA
        query = f'''CREATE TABLE {historico_usuarios} 
            WITH (    
             external_location = 's3://iadbprod-csd-hub-analyticaldata/graphdata-mobility-usuarios/{historico_usuarios}',  
             format = 'PARQUET', 
             parquet_compression = 'SNAPPY',
             partitioned_by = ARRAY['iso_country_code']) AS 
            SELECT *
            FROM {tabla_final_usuario_dia} '''
        cursor.execute(query)
    else:
        # INSERTAR
        query = f'''INSERT INTO {historico_usuarios} SELECT * FROM {tabla_final_usuario_dia} '''
        cursor.execute(query) 
    print(f"Tiempo total dia {fecha}: {int(time.time() - start_time_day)}s")
    print("*"*90)

901
todos_usuarios_delta_dist_corregida_901
Tiempo query select: 6s
Tamaño tabla sin id_provincia en historico: 228551
Pais: AR
(31413, 3)
Pais: CO
(12933, 3)
Pais: PE
(4350, 3)
Pais: NI
(0, 3)
Pais: SV
(1684, 3)
Pais: BO
(4257, 3)
Pais: DO
(2288, 3)
Pais: CL
(13618, 3)
Pais: CR
(1560, 3)
Pais: PY
(3202, 3)
Pais: BZ
(232, 3)
Pais: HN
(2225, 3)
Pais: JM
(3409, 3)
Pais: TT
(1568, 3)
Pais: UY
(0, 3)
Pais: GY
(464, 3)
Pais: EC
(4476, 3)
Pais: GT
(2639, 3)
Pais: PA
(1214, 3)
Pais: VE
(3028, 3)
Pais: BR
(99604, 3)
Pais: MX
(34387, 3)
Asociar id_provincia a hogares: 21s
Guardar tabla base_nuevos_dia:0s
Tiempo total dia 2020-09-01 00:00:00: 32s
******************************************************************************************
Crear tabla final del dia con hogares e id_provincia: 16s
Tiempo total dia 2020-09-01 00:00:00: 193s
******************************************************************************************
902
todos_usuarios_delta_dist_corregida_902
Tiempo query select: 6s
Tama

Tiempo query select: 6s
Tamaño tabla sin id_provincia en historico: 87438
Pais: AR
(6950, 3)
Pais: CO
(5301, 3)
Pais: PE
(3709, 3)
Pais: NI
(0, 3)
Pais: SV
(666, 3)
Pais: BO
(2200, 3)
Pais: DO
(1001, 3)
Pais: CL
(4334, 3)
Pais: CR
(563, 3)
Pais: PY
(1082, 3)
Pais: BZ
(35, 3)
Pais: HN
(626, 3)
Pais: JM
(1223, 3)
Pais: TT
(318, 3)
Pais: UY
(0, 3)
Pais: GY
(198, 3)
Pais: EC
(1759, 3)
Pais: GT
(1032, 3)
Pais: PA
(605, 3)
Pais: VE
(1419, 3)
Pais: BR
(40301, 3)
Pais: MX
(14116, 3)
Asociar id_provincia a hogares: 14s
Guardar tabla base_nuevos_dia:0s
Tiempo total dia 2020-09-10 00:00:00: 24s
******************************************************************************************
Crear tabla final del dia con hogares e id_provincia: 12s
Tiempo total dia 2020-09-10 00:00:00: 180s
******************************************************************************************
911
todos_usuarios_delta_dist_corregida_911
Tiempo query select: 6s
Tamaño tabla sin id_provincia en historico: 86991
Pais: AR

Tiempo query select: 5s
Tamaño tabla sin id_provincia en historico: 21610
Pais: AR
(998, 3)
Pais: CO
(947, 3)
Pais: PE
(671, 3)
Pais: NI
(0, 3)
Pais: SV
(193, 3)
Pais: BO
(686, 3)
Pais: DO
(302, 3)
Pais: CL
(646, 3)
Pais: CR
(155, 3)
Pais: PY
(435, 3)
Pais: BZ
(11, 3)
Pais: HN
(219, 3)
Pais: JM
(349, 3)
Pais: TT
(225, 3)
Pais: UY
(0, 3)
Pais: GY
(45, 3)
Pais: EC
(536, 3)
Pais: GT
(575, 3)
Pais: PA
(201, 3)
Pais: VE
(521, 3)
Pais: BR
(8746, 3)
Pais: MX
(5149, 3)
Asociar id_provincia a hogares: 10s
Guardar tabla base_nuevos_dia:0s
Tiempo total dia 2020-09-19 00:00:00: 20s
******************************************************************************************
Crear tabla final del dia con hogares e id_provincia: 7s
Tiempo total dia 2020-09-19 00:00:00: 167s
******************************************************************************************
920
todos_usuarios_delta_dist_corregida_920
Tiempo query select: 5s
Tamaño tabla sin id_provincia en historico: 34126
Pais: AR
(1865, 3)
Pais

Tiempo query select: 6s
Tamaño tabla sin id_provincia en historico: 55544
Pais: AR
(3147, 3)
Pais: CO
(3477, 3)
Pais: PE
(2999, 3)
Pais: NI
(0, 3)
Pais: SV
(524, 3)
Pais: BO
(1444, 3)
Pais: DO
(797, 3)
Pais: CL
(2742, 3)
Pais: CR
(632, 3)
Pais: PY
(755, 3)
Pais: BZ
(6, 3)
Pais: HN
(645, 3)
Pais: JM
(1028, 3)
Pais: TT
(243, 3)
Pais: UY
(0, 3)
Pais: GY
(100, 3)
Pais: EC
(1512, 3)
Pais: GT
(1426, 3)
Pais: PA
(509, 3)
Pais: VE
(1027, 3)
Pais: BR
(23164, 3)
Pais: MX
(9367, 3)
Asociar id_provincia a hogares: 12s
Guardar tabla base_nuevos_dia:0s
Tiempo total dia 2020-09-28 00:00:00: 23s
******************************************************************************************
Crear tabla final del dia con hogares e id_provincia: 12s
Tiempo total dia 2020-09-28 00:00:00: 176s
******************************************************************************************
929
todos_usuarios_delta_dist_corregida_929
Tiempo query select: 5s
Tamaño tabla sin id_provincia en historico: 50188
Pais: AR
(26